# Drillhole Processing with gslib-zero

This notebook covers drillhole data processing utilities:
1. Minimum curvature desurvey
2. Length-weighted compositing
3. Interval table merging
4. Integration with kriging workflow

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from gslib_zero import (
    desurvey, composite, merge_intervals,
    nscore, gamv, kt3d,
    GridSpec, VariogramModel, SearchParameters
)

## 1. Create Sample Drillhole Data

We'll create synthetic drillhole data with collar, survey, assay, and geology tables.

In [ ]:
np.random.seed(42)

# Collar table - 5 drillholes
collar = pd.DataFrame({
    'holeid': ['DH01', 'DH02', 'DH03', 'DH04', 'DH05'],
    'x': [100.0, 200.0, 300.0, 150.0, 250.0],
    'y': [100.0, 100.0, 100.0, 200.0, 200.0],
    'z': [500.0, 510.0, 505.0, 495.0, 502.0],  # Surface elevation
})

print("Collar Table:")
collar

In [ ]:
# Survey table - measurements at intervals down the hole
# Dip convention: negative = down (e.g., -60 = 60 degrees below horizontal)
survey_records = []
for hole in collar.holeid:
    # Start vertical, gradually deviate
    depths = [0, 25, 50, 75, 100, 125, 150]
    for i, d in enumerate(depths):
        azm = 90.0 + np.random.normal(0, 5)  # Roughly east, with drift
        dip = -75.0 + i * 2 + np.random.normal(0, 2)  # Starts steep, flattens
        survey_records.append({'holeid': hole, 'depth': d, 'azimuth': azm, 'dip': dip})

survey = pd.DataFrame(survey_records)
print(f"Survey records: {len(survey)}")
survey.head(10)

In [ ]:
# Assay table - grade samples at various intervals
assay_records = []
for hole in collar.holeid:
    current = 0.0
    while current < 140:
        length = np.random.choice([1.0, 1.5, 2.0])  # Variable sample lengths
        # Grade varies with depth and has spatial correlation
        hole_idx = list(collar.holeid).index(hole)
        base_grade = 1.5 + 0.01 * current + 0.005 * collar.x.iloc[hole_idx]
        grade = base_grade + np.random.exponential(0.5)
        assay_records.append({
            'holeid': hole,
            'from': current,
            'to': current + length,
            'au_ppm': grade
        })
        current += length

assay = pd.DataFrame(assay_records)
print(f"Assay samples: {len(assay)}")
print(f"Sample lengths: {(assay.to - assay['from']).unique()}")
assay.head(10)

In [ ]:
# Geology table - rock type intervals
geology_records = []
for hole in collar.holeid:
    # Simple geology: overburden -> oxide -> fresh
    geology_records.extend([
        {'holeid': hole, 'from': 0, 'to': 20, 'rocktype': 'OVB'},
        {'holeid': hole, 'from': 20, 'to': 60, 'rocktype': 'OX'},
        {'holeid': hole, 'from': 60, 'to': 150, 'rocktype': 'FR'},
    ])

geology = pd.DataFrame(geology_records)
print("Geology Table:")
geology.head(10)

## 2. Minimum Curvature Desurvey

Convert downhole depths to 3D coordinates using the minimum curvature method.

In [ ]:
# Desurvey at survey stations only
survey_coords = desurvey(
    collar=collar,
    survey=survey,
)

print(f"Survey coordinates: {len(survey_coords)}")
survey_coords.head(10)

In [ ]:
# Desurvey at sample midpoints
sample_coords = desurvey(
    collar=collar,
    survey=survey,
    depths=assay,  # Compute at assay midpoints
)

print(f"Sample coordinates: {len(sample_coords)}")
sample_coords.head(10)

In [ ]:
# Visualize drillhole traces in 3D
fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')

colors = plt.cm.tab10(np.linspace(0, 1, 5))

for i, hole in enumerate(collar.holeid):
    mask = survey_coords.holeid == hole
    ax.plot(
        survey_coords.x[mask],
        survey_coords.y[mask],
        survey_coords.z[mask],
        'o-', color=colors[i], label=hole, markersize=3
    )

ax.set_xlabel('X (Easting)')
ax.set_ylabel('Y (Northing)')
ax.set_zlabel('Z (Elevation)')
ax.set_title('Drillhole Traces (Minimum Curvature Desurvey)')
ax.legend()
ax.view_init(elev=20, azim=-60)
plt.tight_layout()
plt.show()

## 3. Interval Merging

Merge assay and geology tables to get rock type for each assay interval.

In [ ]:
# Merge assay and geology
merged = merge_intervals(assay, geology)

print(f"Merged records: {len(merged)}")
print(f"\nColumns: {list(merged.columns)}")
merged.head(15)

In [ ]:
# Check that boundaries are split correctly
hole1 = merged[merged.holeid == 'DH01'].copy()
print("DH01 intervals near geology boundaries (depth 20 and 60):")
print(hole1[(hole1['from'] < 25) | ((hole1['from'] >= 55) & (hole1['from'] <= 65))])

## 4. Length-Weighted Compositing

Create fixed-length composites, optionally breaking at domain boundaries.

In [ ]:
# Simple compositing without domain control
comp_simple = composite(
    merged,
    length=5.0,
    min_coverage=0.5,
    columns=['au_ppm'],
)

print(f"Simple composites: {len(comp_simple.data)}")
print(f"Mean coverage: {comp_simple.coverage.mean():.2%}")
comp_simple.data.head(10)

In [ ]:
# Compositing with domain breaks
comp_domain = composite(
    merged,
    length=5.0,
    min_coverage=0.5,
    columns=['au_ppm'],
    domain_column='rocktype',
)

print(f"Domain composites: {len(comp_domain.data)}")
print(f"Mean coverage: {comp_domain.coverage.mean():.2%}")

# Show composites near domain boundary
print("\nComposites near depth 60 (OX/FR boundary):")
boundary_comps = comp_domain.data[
    (comp_domain.data['from'] >= 55) & (comp_domain.data['to'] <= 70)
]
boundary_comps

In [ ]:
# Compare original vs composited distributions
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

axes[0].hist(merged.au_ppm, bins=30, alpha=0.7, label='Original')
axes[0].hist(comp_domain.data.au_ppm, bins=30, alpha=0.7, label='Composited')
axes[0].set_xlabel('Au (ppm)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Grade Distribution: Original vs Composited')
axes[0].legend()

# Sample lengths
orig_lengths = merged['to'] - merged['from']
comp_lengths = comp_domain.data['to'] - comp_domain.data['from']
axes[1].hist(orig_lengths, bins=20, alpha=0.7, label='Original')
axes[1].hist(comp_lengths, bins=20, alpha=0.7, label='Composited')
axes[1].set_xlabel('Sample Length (m)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Sample Length Distribution')
axes[1].legend()

plt.tight_layout()
plt.show()

## 5. Prepare Data for Kriging

Combine composites with 3D coordinates for estimation.

In [ ]:
# Get 3D coordinates at composite midpoints
comp_coords = desurvey(
    collar=collar,
    survey=survey,
    depths=comp_domain.data,
)

# Merge coordinates with composite grades
# Match on holeid and depth (from desurvey output)
composites_3d = comp_domain.data.copy()
composites_3d['mid_depth'] = (composites_3d['from'] + composites_3d['to']) / 2

# Add coordinates
composites_3d = composites_3d.merge(
    comp_coords,
    left_on=['holeid', 'mid_depth'],
    right_on=['holeid', 'depth'],
    how='left'
).drop(columns=['depth', 'mid_depth'])

print(f"Composites with 3D coordinates: {len(composites_3d)}")
composites_3d.head(10)

In [ ]:
# Filter to "Fresh" rock type for domain estimation
fresh_data = composites_3d[composites_3d.rocktype == 'FR'].copy()
print(f"Fresh rock composites: {len(fresh_data)}")
print(f"Grade range: {fresh_data.au_ppm.min():.2f} - {fresh_data.au_ppm.max():.2f}")
print(f"Mean grade: {fresh_data.au_ppm.mean():.2f}")

In [ ]:
# Normal score transform
fresh_data['nscore'], transform_table = nscore(fresh_data.au_ppm, binary=True)

print(f"Normal score mean: {fresh_data.nscore.mean():.4f}")
print(f"Normal score std: {fresh_data.nscore.std():.4f}")

## 6. Variogram Analysis

In [ ]:
# Compute experimental variogram
vario_result = gamv(
    fresh_data.x, fresh_data.y, fresh_data.z, fresh_data.nscore,
    nlag=10,
    lag_distance=20.0,
    binary=True
)[0]

# Plot
plt.figure(figsize=(8, 5))
plt.plot(vario_result.lag_distances, vario_result.gamma, 'bo-', label='Experimental')
plt.axhline(1.0, color='gray', linestyle='--', alpha=0.5, label='Sill = 1.0')
plt.xlabel('Lag Distance (m)')
plt.ylabel('Gamma')
plt.title('Experimental Variogram - Fresh Rock Composites')
plt.legend()
plt.xlim(0, None)
plt.ylim(0, None)
plt.show()

In [ ]:
# Fit variogram model
variogram = VariogramModel.spherical(
    sill=0.8,
    ranges=(80, 80, 40),  # Horizontal, horizontal, vertical
    nugget=0.2
)

print(f"Total sill: {variogram.total_sill}")
print(f"Nugget: {variogram.nugget}")

## 7. Kriging Estimation

In [ ]:
# Define grid covering the data extent
grid = GridSpec(
    nx=20, ny=15, nz=10,
    xmin=fresh_data.x.min() - 10,
    ymin=fresh_data.y.min() - 10,
    zmin=fresh_data.z.min() - 5,
    xsiz=10, ysiz=10, zsiz=10,
)

print(f"Grid cells: {grid.ncells}")
print(f"X range: {grid.xmin:.0f} to {grid.xmax:.0f}")
print(f"Y range: {grid.ymin:.0f} to {grid.ymax:.0f}")
print(f"Z range: {grid.zmin:.0f} to {grid.zmax:.0f}")

In [ ]:
# Search parameters
search = SearchParameters(
    radius1=150,
    radius2=150,
    radius3=100,
    min_samples=3,
    max_samples=12,
)

# Run kriging - using DataFrame pattern
result = kt3d(
    data=fresh_data, value_col='nscore',
    grid=grid, variogram=variogram, search=search,
    kriging_type='ordinary',
    binary=True
)

print(f"Estimate shape: {result.estimate.shape}")
print(f"Valid estimates: {(result.estimate > -998).sum()}")

In [ ]:
# Visualize a horizontal slice
z_slice = 5  # Middle z-level

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Estimates
estimate_slice = result.estimate[z_slice]
estimate_slice[estimate_slice < -998] = np.nan  # Mask unestimated

im1 = axes[0].imshow(
    estimate_slice,
    extent=[grid.xmin, grid.xmax, grid.ymin, grid.ymax],
    origin='lower',
    cmap='viridis'
)
# Show sample locations at this level
z_center = grid.zmin + (z_slice + 0.5) * grid.zsiz
near_z = fresh_data[abs(fresh_data.z - z_center) < grid.zsiz]
axes[0].scatter(near_z.x, near_z.y, c='red', s=30, alpha=0.7, label='Samples')
axes[0].set_xlabel('X')
axes[0].set_ylabel('Y')
axes[0].set_title(f'Kriged Estimates (Z = {z_center:.0f})')
axes[0].legend()
plt.colorbar(im1, ax=axes[0], label='Normal Score')

# Variance
variance_slice = result.variance[z_slice]
variance_slice[variance_slice < 0] = np.nan

im2 = axes[1].imshow(
    variance_slice,
    extent=[grid.xmin, grid.xmax, grid.ymin, grid.ymax],
    origin='lower',
    cmap='Reds'
)
axes[1].scatter(near_z.x, near_z.y, c='blue', s=30, alpha=0.7, label='Samples')
axes[1].set_xlabel('X')
axes[1].set_ylabel('Y')
axes[1].set_title(f'Kriging Variance (Z = {z_center:.0f})')
axes[1].legend()
plt.colorbar(im2, ax=axes[1], label='Variance')

plt.tight_layout()
plt.show()

## Summary

This notebook demonstrated the drillhole-to-estimate workflow:

1. **Desurvey**: Convert downhole depths to 3D coordinates using minimum curvature
2. **Merge intervals**: Combine assay and geology tables, splitting at boundaries
3. **Composite**: Create fixed-length composites with domain control
4. **Prepare for estimation**: Add 3D coordinates to composites, filter by domain
5. **Variogram + Kriging**: Standard geostatistical workflow with the processed data

Key functions used:
- `desurvey()` - Minimum curvature 3D coordinate calculation
- `merge_intervals()` - Combine multiple interval tables
- `composite()` - Length-weighted compositing with domain breaks